## Step 1: Load player stats from SQLite database

In this step, we connect to the SQLite database where all our ESPN-scraped player data (from 2002 to 2025) is stored.  
We load the `player_season_stats` table into a pandas DataFrame called `df` so we can work with it.

In [2]:
import sqlite3
import pandas as pd

# Connect to your SQLite database
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")

# Load the player season stats table into a DataFrame
df = pd.read_sql("SELECT * FROM player_season_stats", conn)

# Close the database connection
conn.close()

# Preview the first few rows
df.head()

,RK,Name,POS,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FTA,FT%,REB,AST,STL,BLK,TO,DD2,TD3,season
0,-,Allen IversonPHI,SG,60,43.7,31.4,11.1,27.8,39.8,1.3,...,9.8,81.2,4.5,5.5,2.8,0.2,4.0,4,1,2002
1,-,Shaquille O'NealLAL,C,67,36.1,27.2,10.6,18.3,57.9,0.0,...,10.7,55.5,10.7,3.0,0.6,2.0,2.6,40,0,2002
2,-,Paul PierceBOS,SF,82,40.3,26.1,8.6,19.5,44.2,2.6,...,7.8,80.9,6.9,3.2,1.9,1.0,2.9,17,0,2002
3,-,Tracy McGradyORL,SG,76,38.3,25.6,9.4,20.9,45.1,1.4,...,7.3,74.8,7.9,5.3,1.6,1.0,2.5,24,1,2002
4,-,Tim DuncanSA,C,82,40.6,25.5,9.3,18.3,50.8,0.0,...,8.5,79.9,12.7,3.7,0.7,2.5,3.2,67,0,2002


## Step 2A: Split player_name and team_name from Name column

In the raw data, the `Name` column combines both the player’s name and their team abbreviation (e.g., "Allen IversonPHI").  
We split this into two new columns:
- `player_name`: Just the player's name
- `team_name`: 2–3 letter team abbreviation (used later to match team stats)


In [3]:
df['team_name'] = df['Name'].str.extract(r'([A-Z]{2,3})$', expand=False)
df['player_name'] = df['Name'].str.replace(r'([A-Z]{2,3})$', '', regex=True).str.strip()
df[['Name', 'player_name', 'team_name']].head(10)


,Name,player_name,team_name
0,Allen IversonPHI,Allen Iverson,PHI
1,Shaquille O'NealLAL,Shaquille O'Neal,LAL
2,Paul PierceBOS,Paul Pierce,BOS
3,Tracy McGradyORL,Tracy McGrady,ORL
4,Tim DuncanSA,Tim Duncan,SA
5,Kobe BryantLAL,Kobe Bryant,LAL
6,Vince CarterTOR,Vince Carter,TOR
7,Chris WebberSAC,Chris Webber,SAC
8,Dirk NowitzkiDAL,Dirk Nowitzki,DAL
9,Michael JordanWSH,Michael Jordan,WSH


## Step 2B: Create unique player_id for each player

To track players across seasons, we assign each unique `player_name` a consistent `player_id`.  
This ID stays the same no matter which season or team the player is in.  
It helps with merging, filtering, and modeling later on.


In [5]:
# Create a unique player ID for each player name
df['player_id'] = df['player_name'].astype('category').cat.codes


## Step 2C: Clean up player_name to fix suffix issues

Some player names include extra info like "DEN/", "NY/", or stray letters that came from inconsistent scraping.  
We clean these from the `player_name` column using a regex pattern.  
After cleaning, we **reassign the `player_id`s** so each player has one consistent ID across all rows.


In [7]:
df['player_name'] = df['player_name'].str.replace(r'[A-Z]{2,3}/?$', '', regex=True).str.strip()
df['player_id'] = df['player_name'].astype('category').cat.codes


In [8]:
df[['player_name', 'player_id']].drop_duplicates().sort_values('player_id').head(15)


,player_name,player_id
421,Aaron Brooks,0
842,Aaron Gordon,1
192,Al Harrington,2
580,Al Horford,3
321,Al Jefferson,4
477,Al JeffersonU,5
25,Allan Houston,6
0,Allen Iverson,7
47,Alonzo Mourning,8
1131,Alperen Sengun,9


## 🔐 Step 1: Save `player_id` mapping to SQLite and CSV

We'll store the cleaned `player_id` and `player_name` pairs for future merging.  
Primary method: save to SQLite for centralized access.  
Backup: optional CSV export for easier viewing/debugging.


In [9]:
import sqlite3

# Create mapping table
player_map = df[['player_id', 'player_name']].drop_duplicates().sort_values('player_id')

# Save to SQLite
conn = sqlite3.connect('../data/sqlite/nba_mvp.db')
player_map.to_sql('player_id_map', conn, if_exists='replace', index=False)
conn.close()

print("✅ Saved player_id_map to SQLite!")


✅ Saved player_id_map to SQLite!


### 🏆 Add Team Win Percentage and Rank
We will calculate each NBA team's win percentage and rank per season using game data, then merge it into the player dataset as new features: `team_win_pct` and `team_rank`.


In [10]:
# Example: Mock team records per season
team_records = pd.DataFrame({
    'season': [2002, 2002, 2002, 2002],
    'team_name': ['PHI', 'LAL', 'SAC', 'DAL'],
    'wins': [43, 58, 61, 57],
    'losses': [39, 24, 21, 25]
})

# Step 1: Calculate win percentage
team_records['team_win_pct'] = team_records['wins'] / (team_records['wins'] + team_records['losses'])

# Step 2: Rank teams by win percentage (per season)
team_records['team_rank'] = team_records.groupby('season')['team_win_pct'].rank(ascending=False, method='dense')

# Step 3: Merge with player DataFrame
df = df.merge(team_records[['season', 'team_name', 'team_win_pct', 'team_rank']], on=['season', 'team_name'], how='left')


## 💾 Save Engineered Features to SQLite

We save our updated DataFrame with added features (e.g., player_id, team_name, etc.) to a new table called `engineered_features`. This allows future steps (model training, evaluation) to use clean and enriched data directly from the database.


In [11]:
import sqlite3

# Connect to the same database file
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")

# Save DataFrame to new table
df.to_sql("engineered_features", conn, if_exists="replace", index=False)

conn.close()

print("✅ Saved engineered_features to SQLite!")


✅ Saved engineered_features to SQLite!


### 🔍 Check for Existing Tables in SQLite
This checks what data already exists in our database, so we don't duplicate work.


In [12]:
import sqlite3

# Connect to the database
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")
cursor = conn.cursor()

# Show all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print each table name
for table in tables:
    print("📁", table[0])

conn.close()


📁 player_season_stats
📁 player_id_map
📁 engineered_features


In [14]:
import sqlite3
import pandas as pd

# Connect to your SQLite database
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")

# Show all existing table names
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = pd.read_sql(query, conn)
conn.close()

tables_df


,name
0,player_season_stats
1,player_id_map
2,engineered_features


## 🧩 Step 1: Load Data from SQLite

We begin by loading the two relevant tables from our SQLite database:
- `player_season_stats`: contains ESPN player stats from 2002–2025.
- `engineered_features`: our current working feature set for MVP prediction.

This allows us to later merge these datasets based on player name and season.


In [15]:
import sqlite3
import pandas as pd

# Load from database
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")

player_stats = pd.read_sql("SELECT * FROM player_season_stats", conn)
engineered_features = pd.read_sql("SELECT * FROM engineered_features", conn)

conn.close()

print("📊 Loaded player stats:", player_stats.shape)
print("📊 Loaded existing features:", engineered_features.shape)


📊 Loaded player stats: (1150, 23)
📊 Loaded existing features: (1150, 28)


## 🧩 Step 2: Inspect Column Names

We need to verify that both tables share common columns — specifically `Name` and `season` — which we’ll use as keys when merging.

This ensures that we’re aligning the stats with the correct player and year in the feature set.


In [16]:
print(player_stats.columns)
print(engineered_features.columns)


Index(['RK', 'Name', 'POS', 'GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM',
       '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'REB', 'AST', 'STL', 'BLK', 'TO',
       'DD2', 'TD3', 'season'],
      dtype='object')
Index(['RK', 'Name', 'POS', 'GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM',
       '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'REB', 'AST', 'STL', 'BLK', 'TO',
       'DD2', 'TD3', 'season', 'team_name', 'player_name', 'player_id',
       'team_win_pct', 'team_rank'],
      dtype='object')


## 🧩 Step 3: Clean & Merge Tables

Now we’ll clean the `Name` columns and merge `player_season_stats` with `engineered_features` using the common keys:
- `Name` (from ESPN player stats)
- `season` (shared across both)

We'll standardize the `Name` field formatting to avoid mismatches (e.g., trimming whitespace), then join the tables to produce a single enriched dataset.


In [17]:
import sqlite3
import pandas as pd

# Connect to DB and load tables
conn = sqlite3.connect("../data/sqlite/nba_mvp.db")
player_stats = pd.read_sql("SELECT * FROM player_season_stats", conn)
features = pd.read_sql("SELECT * FROM engineered_features", conn)

# Clean up player names for both tables
player_stats["Name"] = player_stats["Name"].str.strip()
features["Name"] = features["Name"].str.strip()

# Merge on Name + season
merged = pd.merge(player_stats, features, on=["Name", "season"], how="inner")

# Check result
print("✅ Merged shape:", merged.shape)
print(merged[["Name", "season", "team_name", "team_win_pct", "team_rank"]].head())

# (Optional) Save to new table in SQLite
merged.to_sql("final_player_data", conn, if_exists="replace", index=False)
conn.close()
print("📦 Merged data saved as 'final_player_data'")


✅ Merged shape: (1150, 49)
                  Name  season team_name  team_win_pct  team_rank
0     Allen IversonPHI    2002       PHI      0.524390        4.0
1  Shaquille O'NealLAL    2002       LAL      0.707317        2.0
2       Paul PierceBOS    2002       BOS           NaN        NaN
3     Tracy McGradyORL    2002       ORL           NaN        NaN
4         Tim DuncanSA    2002        SA           NaN        NaN
📦 Merged data saved as 'final_player_data'
